In [1]:
import duckdb
from datetime import datetime

table_name2dtypes = {
    "agency": {
        "agency_id": "varchar",
        "agency_name": "varchar",
        "agency_url": "varchar",
        "agency_timezone": "varchar",
        "agency_lang": "varchar",
        "agency_phone": "varchar",
    },
    "calendar": {
        "service_id": "varchar",
        "monday": "int",
        "tuesday": "int",
        "wednesday": "int",
        "thursday": "int",
        "friday": "int",
        "saturday": "int",
        "sunday": "int",
        "start_date": "int",
        "end_date": "int",
    },
    "calendar_dates": {
        "service_id": "varchar",
        "date": "int",
        "exception_type": "int",
    },
    "feed_info": {
        "feed_publisher_name": "varchar",
        "feed_publisher_url": "varchar",
        "feed_lang": "varchar",
        "feed_start_date": "varchar",
        "feed_end_date": "varchar",
        "feed_version": "varchar",
    },
    "routes": {
        "route_id": "varchar",
        "agency_id": "varchar",
        "route_short_name": "varchar",
        "route_long_name": "varchar",
        "route_desc": "varchar",
        "route_type": "varchar",
    },
    "stops": {
        "stop_id": "varchar",
        "stop_name": "varchar",
        "stop_lat": "float",
        "stop_lon": "float",
        "location_type": "varchar",
        "parent_station": "varchar",
    },
    "stop_times": {
        "trip_id": "varchar",
        #"arrival_time": "varchar",
        #"departure_time": "varchar",
        "arrival_time": "time",
        "departure_time": "time",
        "stop_id":  "varchar",
        "stop_sequence": "int",
        "pickup_type": "varchar",
        "drop_off_type": "varchar",
    },
    "trips": {
        "route_id": "varchar",
        "service_id": "varchar",
        "trip_id": "varchar",
        "trip_headsign": "varchar",
        "trip_short_name": "varchar",
        "direction_id": "varchar",
        "block_id": "varchar",
    },
    "transfers": {
        "from_stop_id": "varchar",
        "to_stop_id": "varchar",
        "transfer_type": "varchar",
        "min_transfer_time": "varchar",
    },
}

In [2]:
base_dir = '../../data/gtfs/20240711/'
db_path = '/home/raph/data/gtfs/20240923.ddb'

In [115]:
with duckdb.connect(db_path) as c:
    c.sql(f"create table transport_name_mapping as select * from read_csv('{'./assets/transport_name_mapping.csv'}')")
    for table_name, dtypes in table_name2dtypes.items():
        c.sql(f"create table {table_name} as select * from read_csv('{base_dir+table_name+'.txt'}', dtypes={list(dtypes.values())}, ignore_errors=True)")


In [3]:
local_stops_ids = tuple([
    "8501124:0:1", # Cully, voie 1?
    "8501124:0:2", # Cully, voie 2?
    "8501124:0:3", # Cully, voie 3?
    "8570559", # Cully gare Post?
    "8501125:0:1", # Epesses CFF, voie 1?
    "8501125:0:2", # Epesses CFF, voie 2?
    "8510137", # Epesses, gare Post?
    "8570561", # Epesses, village
])

conn = duckdb.connect(db_path)

In [4]:
today = datetime.now()
today_int = int(today.strftime("%Y%m%d"))
weekday_str = today.strftime("%A").lower()

In [5]:
date = datetime.now()
stop_ids = local_stops_ids
date_yyymmdd = int(date.strftime("%Y%m%d"))
date_yy = int(date.strftime("%Y"))
date_mm = int(date.strftime("%m"))
date_dd = int(date.strftime("%d"))
weekday_str = date.strftime("%A").lower()
stop_ids_filter = "" if len(stop_ids) == 0 else f"AND stop_times.stop_id in {str(tuple(stop_ids))}"
with duckdb.connect(db_path) as db:
#                JOIN (
#                    SELECT stop_times.trip_id, last_stop_sequence, stop_name as last_stop_name
#                    FROM (
#                        SELECT trip_id, MAX(stop_sequence) as last_stop_sequence
#                        FROM stop_times
#                        GROUP BY trip_id
#                    ) as max_sequence
#                    INNER JOIN stop_times ON (max_sequence.trip_id = stop_times.trip_id AND max_sequence.last_stop_sequence = stop_times.stop_sequence)
#                    INNER JOIN stops ON (stops.stop_id = stop_times.stop_id)
#                ) as last_stop ON (last_stop.trip_id = stop_times.trip_id)
#
            #"""SELECT stop_times.trip_id, stop_times.stop_id, stop_times.stop_sequence, routes.route_short_name, stops.stop_name, trips.trip_headsign, stop_times.arrival_time, stop_times.departure_time, transport_name_mapping.FR"""
    print(db.execute(f"""
        SELECT  *,
                CASE
                    WHEN regexp_full_match(departure_time, '^([01]?[0-9]|2[0-4]):[0-5][0-9]:[0-5][0-9]$')
                    THEN make_timestamp({date_yy}, {date_mm}, {date_dd},
                        cast(split_part(departure_time, ':', 1) as INTEGER) % 24,
                        cast(split_part(departure_time, ':', 2) as INTEGER),
                        cast(split_part(departure_time, ':', 3) AS INTEGER))
                    ELSE NULL
                END as departure_timestamp,
                CASE
                    WHEN regexp_full_match(arrival_time, '^([01]?[0-9]|2[0-4]):[0-5][0-9]:[0-5][0-9]$')
                    THEN make_timestamp({date_yy}, {date_mm}, {date_dd},
                        cast(split_part(arrival_time, ':', 1) as INTEGER) % 24,
                        cast(split_part(arrival_time, ':', 2) as INTEGER),
                        cast(split_part(arrival_time, ':', 3) AS INTEGER))
                    ELSE NULL
                END as arrival_timestamp
        FROM stop_times
        JOIN stops ON (stops.stop_id = stop_times.stop_id)
        JOIN trips ON (trips.trip_id = stop_times.trip_id)
        JOIN routes on (routes.route_id = trips.route_id)
        JOIN calendar ON (calendar.service_id = trips.service_id)
        JOIN (
             SELECT trip_id, MAX(stop_sequence) as last_stop_sequence
             FROM stop_times
             GROUP BY trip_id
        ) as last_stop ON (last_stop.trip_id = stop_times.trip_id)
        JOIN transport_name_mapping ON (transport_name_mapping.Abbreviation = routes.route_desc)
        WHERE
        stop_times.stop_sequence != last_stop.last_stop_sequence
        {stop_ids_filter}
        AND (
                (
                calendar.{weekday_str} = 1
                AND calendar.start_date < {date_yyymmdd}
                AND calendar.end_date > {date_yyymmdd}
                )
            OR (
                trips.service_id IN (
                    SELECT service_id
                    FROM calendar_dates
                    WHERE date={date_yyymmdd}
                    AND exception_type = 1
                    )
                )
            )
        AND trips.service_id NOT IN (
            SELECT service_id
            FROM calendar_dates
            WHERE date={date_yyymmdd}
            AND exception_type = 2)
        ORDER BY departure_timestamp, departure_time
        """))


In [6]:
conn.sql("""
         SELECT *
         FROM stop_times
         JOIN (
             SELECT stop_times.trip_id as trip_id, last_stop_sequence, stop_name as last_stop_name
             FROM (
                 SELECT trip_id, MAX(stop_sequence) as last_stop_sequence
                 FROM stop_times
                 GROUP BY trip_id
             ) as max_sequence
             INNER JOIN stop_times ON (max_sequence.trip_id = stop_times.trip_id AND max_sequence.last_stop_sequence = stop_times.stop_sequence)
             INNER JOIN stops ON (stops.stop_id = stop_times.stop_id)
         ) as last_stop ON (last_stop.trip_id = stop_times.trip_id)
         """)

┌──────────────────────┬──────────────┬────────────────┬───┬────────────────────┬──────────────────────┐
│       trip_id        │ arrival_time │ departure_time │ … │ last_stop_sequence │    last_stop_name    │
│       varchar        │   varchar    │    varchar     │   │       int32        │       varchar        │
├──────────────────────┼──────────────┼────────────────┼───┼────────────────────┼──────────────────────┤
│ 1210.TA.91-9-j24-1…  │ 20:08:00     │ 20:08:00       │ … │                 20 │ Bern Wankdorf, Bah…  │
│ 1210.TA.91-m1-j24-…  │ 23:14:00     │ 23:14:00       │ … │                 10 │ Ecublens VD, EPFL    │
│ 1210.TA.91-m2-j24-…  │ 06:13:00     │ 06:13:00       │ … │                 14 │ Epalinges, Croiset…  │
│ 12100.TA.91-12-B-j…  │ 19:04:00     │ 19:04:00       │ … │                 25 │ Lancy-Bachet, gare   │
│ 12100.TA.91-14-D-j…  │ 22:29:00     │ 22:29:00       │ … │                 30 │ Bernex, Vailly       │
│ 12100.TA.91-15-D-j…  │ 16:01:00     │ 16:01:00       

In [5]:
query = conn.sql(f"SELECT * FROM calendar_dates where date={today_int}")

In [148]:
res = conn.execute(f"""
SELECT routes.route_short_name, stops.stop_name, trips.trip_headsign, stop_times.arrival_time, stop_times.departure_time, transport_name_mapping.FR
FROM stop_times
JOIN stops ON (stops.stop_id = stop_times.stop_id)
JOIN trips ON (trips.trip_id = stop_times.trip_id)
JOIN routes on (routes.route_id = trips.route_id)
JOIN calendar ON (calendar.service_id = trips.service_id)
JOIN (
    SELECT trip_id, MAX(stop_sequence) as max_stop_sequence
    FROM stop_times
    GROUP BY trip_id
) as max_sequence ON (max_sequence.trip_id = stop_times.trip_id)
JOIN transport_name_mapping ON (transport_name_mapping.Abbreviation = routes.route_desc)
JOIN (SELECT make_time(date_part('hour', current_timestamp), date_part('minute', current_timestamp), date_part('second', current_timestamp)) as now
) as time ON TRUE
WHERE
stop_times.stop_id in {local_stops_ids}
AND stop_times.stop_sequence != max_sequence.max_stop_sequence
AND (
        (
        calendar.{weekday_str} = 1
        AND calendar.start_date < {today_int}
        AND calendar.end_date > {today_int}
        )
    OR (
        trips.service_id IN (
            SELECT service_id
            FROM calendar_dates
            WHERE date={today_int}
            AND exception_type = 1
            )
        )
    )
AND trips.service_id NOT IN (
    SELECT service_id
    FROM calendar_dates
    WHERE date={today_int}
    AND exception_type = 2)
AND stop_times.departure_time >= time.now
AND stop_times.departure_time <= time.now + interval '10 minutes'
ORDER BY departure_time
""").df()

res

,route_short_name,stop_name,trip_headsign,arrival_time,departure_time,FR
0,R3,Cully,Aigle,21:37:00,21:37:00,Train régional
1,R3,Epesses,Aigle,21:39:00,21:39:00,Train régional


In [147]:
res.groupby('stop_name').head(5)

,route_short_name,stop_name,trip_headsign,arrival_time,departure_time,FR
0,R3,Epesses,Vallorbe,20:16:00,20:16:00,Train régional
1,R3,Cully,Vallorbe,20:18:00,20:18:00,Train régional
2,R3,Cully,Aigle,20:37:00,20:37:00,Train régional
3,R3,Epesses,Aigle,20:39:00,20:39:00,Train régional
4,R4,Cully,Le Day,20:50:00,20:50:00,Train régional
5,R4,Cully,Aigle,21:09:00,21:09:00,Train régional
6,R3,Epesses,Vallorbe,21:16:00,21:16:00,Train régional
7,R3,Cully,Vallorbe,21:18:00,21:18:00,Train régional
9,R3,Epesses,Aigle,21:39:00,21:39:00,Train régional
12,R3,Epesses,Vallorbe,22:16:00,22:16:00,Train régional


In [85]:
conn.close()

In [31]:
conn.sql(f"SELECT DISTINCT trip_id FROM stop_times where stop_id in {local_stops_ids}")

┌────────────────────────────┐
│          trip_id           │
│          varchar           │
├────────────────────────────┤
│ 1093.TA.91-4-L-j24-1.646.R │
│ 1094.TA.91-4-L-j24-1.646.R │
│ 1095.TA.91-3-T-j24-1.146.R │
│ 1097.TA.91-3-T-j24-1.702.R │
│ 1099.TA.91-3-T-j24-1.61.R  │
│ 110.TA.91-41-Y-j24-1.282.R │
│ 1102.TA.91-3-T-j24-1.186.R │
│ 1105.TA.91-3-T-j24-1.606.R │
│ 1105.TA.91-4-L-j24-1.738.R │
│ 1114.TA.91-3-T-j24-1.870.R │
│            ·               │
│            ·               │
│            ·               │
│ 38.TA.92-6H-Y-j24-1.7.H    │
│ 42.TA.92-6H-Y-j24-1.7.H    │
│ 53.TA.92-6H-Y-j24-1.8.R    │
│ 59.TA.92-6H-Y-j24-1.8.R    │
│ 73.TA.92-6H-Y-j24-1.8.R    │
│ 79.TA.92-6H-Y-j24-1.8.R    │
│ 53.TA.96-206-j24-1.22.H    │
│ 54.TA.96-206-j24-1.22.H    │
│ 55.TA.96-203-j24-1.16.R    │
│ 56.TA.96-206-j24-1.22.H    │
├────────────────────────────┤
│    2349 rows (20 shown)    │
└────────────────────────────┘

In [28]:
conn.table("trips")

┌───────────────┬────────────┬───────────────────────┬─────────────────────┬─────────────────┬──────────────┬──────────┐
│   route_id    │ service_id │        trip_id        │    trip_headsign    │ trip_short_name │ direction_id │ block_id │
│    varchar    │  varchar   │        varchar        │       varchar       │     varchar     │   varchar    │ varchar  │
├───────────────┼────────────┼───────────────────────┼─────────────────────┼─────────────────┼──────────────┼──────────┤
│ 91-10-A-j24-1 │ TA+p60e0   │ 1.TA.91-10-A-j24-1.…  │ Zürich HB SZU       │ 12894           │ 0            │ NULL     │
│ 91-10-A-j24-1 │ TA+p60e0   │ 10.TA.91-10-A-j24-1…  │ Zürich HB SZU       │ 12888           │ 0            │ NULL     │
│ 91-10-A-j24-1 │ TA+hvzd0   │ 100.TA.91-10-A-j24-…  │ Zürich HB SZU       │ 12784           │ 0            │ NULL     │
│ 91-10-A-j24-1 │ TA+hvzd0   │ 101.TA.91-10-A-j24-…  │ Zürich HB SZU       │ 12920           │ 0            │ NULL     │
│ 91-10-A-j24-1 │ TA+p60e0   │ 1

In [15]:
conn.table('stop_times')

┌─────────────────────────┬──────────────┬────────────────┬──────────────┬───────────────┬─────────────┬───────────────┐
│         trip_id         │ arrival_time │ departure_time │   stop_id    │ stop_sequence │ pickup_type │ drop_off_type │
│         varchar         │   varchar    │    varchar     │   varchar    │     int32     │   varchar   │    varchar    │
├─────────────────────────┼──────────────┼────────────────┼──────────────┼───────────────┼─────────────┼───────────────┤
│ 1.TA.91-10-A-j24-1.1.H  │ 17:27:00     │ 17:27:00       │ 8503054:0:1  │             1 │ 0           │ 0             │
│ 1.TA.91-10-A-j24-1.1.H  │ 17:28:00     │ 17:28:00       │ 8503053:0:1  │             2 │ 0           │ 0             │
│ 1.TA.91-10-A-j24-1.1.H  │ 17:29:00     │ 17:29:00       │ 8503052:0:1  │             3 │ 0           │ 0             │
│ 1.TA.91-10-A-j24-1.1.H  │ 17:31:00     │ 17:32:00       │ 8503051:0:1  │             4 │ 0           │ 0             │
│ 1.TA.91-10-A-j24-1.1.H  │ 17:3

In [14]:
conn.sql(f"SELECT * FROM calendar WHERE {weekday_str} = 1 AND start_date < {today_int} AND end_date > {today_int}")

┌────────────┬────────┬─────────┬───────────┬──────────┬────────┬──────────┬────────┬────────────┬──────────┐
│ service_id │ monday │ tuesday │ wednesday │ thursday │ friday │ saturday │ sunday │ start_date │ end_date │
│  varchar   │ int32  │  int32  │   int32   │  int32   │ int32  │  int32   │ int32  │   int32    │  int32   │
├────────────┼────────┼─────────┼───────────┼──────────┼────────┼──────────┼────────┼────────────┼──────────┤
│ TA         │      1 │       1 │         1 │        1 │      1 │        1 │      1 │   20231210 │ 20241214 │
│ TA+00080   │      1 │       0 │         0 │        0 │      0 │        0 │      1 │   20231210 │ 20241214 │
│ TA+00140   │      0 │       0 │         0 │        0 │      1 │        1 │      1 │   20231210 │ 20241214 │
│ TA+00180   │      1 │       1 │         1 │        1 │      0 │        1 │      1 │   20231210 │ 20241214 │
│ TA+00380   │      1 │       1 │         1 │        1 │      0 │        0 │      1 │   20231210 │ 20241214 │
│ TA+00790

In [8]:
conn.execute(f"SELECT * FROM stop_times where stop_id in {local_stops_ids}").df()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type
0,1091.TA.91-3-T-j24-1.700.R,23:37:00,23:37:00,8501124:0:1,14,0,0
1,1091.TA.91-3-T-j24-1.700.R,23:39:00,23:39:00,8501125:0:1,15,0,0
2,1091.TA.91-4-L-j24-1.645.R,22:09:00,22:09:00,8501124:0:1,13,0,0
3,1092.TA.91-3-T-j24-1.145.R,22:41:00,22:42:00,8501124:0:1,14,0,0
4,1092.TA.91-3-T-j24-1.145.R,22:43:00,22:44:00,8501125:0:1,15,0,0
...,...,...,...,...,...,...,...
3564,6.TA.96-203-j24-1.2.H,12:55:00,12:55:00,8570561,4,0,0
3565,6.TA.96-206-j24-1.1.H,13:54:00,13:54:00,8570559,1,0,0
3566,6.TA.96-206-j24-1.1.H,14:05:00,14:05:00,8570559,3,0,0
3567,60.TA.96-203-j24-1.16.R,06:43:00,06:43:00,8570559,16,0,0


In [40]:
str(local_stops_ids)

"('8501124:0:1', '8501124:0:2', '8501124:0:3', '8570559', '8501125:0:1', '8501125:0:2', '8510137', '8570561')"

ParserException: Parser Error: syntax error at or near "select"